In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)


In [4]:
wiki.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model='llama3'))
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
/tmp/ipykernel_3706/3428277755.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model='llama3'))


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f2f935647c0>, search_kwargs={})

In [6]:
from langchain_core.tools import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [7]:
retriever_tool.name

'langsmith_search'

In [8]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
tools=[wiki,arxiv,retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/shreyan25/conda/envs/lang/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f2fc4e4ea70>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community

In [48]:
from langchain_community.chat_models import ChatOllama

## Load Ollama LAMA2 LLM model
llm = ChatOllama(model="llama3", temperature=0)
llm

ChatOllama(model='llama3', temperature=0.0)

In [49]:
from langchain import hub
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [50]:
### Agents

from langchain.agents import  create_react_agent

agent=create_react_agent(llm,tools,prompt_react)

In [51]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OllamaEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f2f935647c0>, search_kw

In [52]:
result  =agent_executor.invoke({"input":"Tell me about Langsmith"})
result



> Entering new AgentExecutor chain...
Thought: Since the question is specifically about LangSmith, I should use the langsmith_search tool.

Action: langsmith_search
Action Input: 'str'Get started by creating your first prompt.
Iterate on models and prompts using the Playground.
Manage prompts programmatically in your application.
Was this page helpful?You can leave detailed feedback on GitHub.NextQuick StartObservabilityEvalsPrompt EngineeringCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.

Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with La

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.'}

In [54]:
generated_answer = result['output']

In [55]:
from sacrebleu.metrics import BLEU
from rouge import Rouge
from sentence_transformers import SentenceTransformer, util

def evaluate_answer(generated_answer, reference_answer):
    """
    Evaluates the generated answer against a reference answer using:
      - BLEU (with sacreBLEU)
      - ROUGE (using the Rouge package)
      - Semantic similarity (via SentenceTransformer)
    """
    # --- BLEU Score using sacreBLEU ---
    bleu_scorer = BLEU()
    bleu_score = bleu_scorer.sentence_score(
        hypothesis=generated_answer,
        references=[reference_answer],
  
    ).score / 100 
    
    #ROUGE Score 
    rouge_scorer = Rouge()
    rouge_scores = rouge_scorer.get_scores(hyps=generated_answer, refs=reference_answer)[0]
    rouge_l_f = rouge_scores["rouge-l"]["f"]
    
    #Semantic Similarity ---
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode([generated_answer, reference_answer], convert_to_tensor=True)
    cosine_sim = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    
    return {
        "bleu": bleu_score,
        "rouge": rouge_l_f,
        "semantic_similarity": cosine_sim
    }


In [56]:
# Example usage:
generated_answer = generated_answer
reference_answer = "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence."

evaluation_results = evaluate_answer(generated_answer, reference_answer)

print("Evaluation Metrics:")
print("BLEU Score:", evaluation_results["bleu"])
print("ROUGE-L F1 Score:", evaluation_results["rouge"])
print("Semantic Similarity:", evaluation_results["semantic_similarity"])

Evaluation Metrics:
BLEU Score: 1.0000000000000004
ROUGE-L F1 Score: 0.999999995
Semantic Similarity: 1.0
